In [55]:
import pandas as pd

from difflib import SequenceMatcher

In [56]:
filename = 'outlets.csv'
data = pd.read_csv(filename, sep='|', header=None)

In [57]:
data.drop(data.columns[[0]], axis='columns', inplace=True)
data.columns = ['Город дистрибьютора', 'Торг_точка_грязная', 'Торг_точка_грязная_адрес', 'outlet_clean_id']

In [58]:
data.tail(5)

,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
20203,Ростов-на-Дону,Яровой А.Т.ИП,"346535, Ростовская обл, Шахты г, Кошевого ул, ...",\N
20204,Ростов-на-Дону,"Яровой В.Т. ИП м-н ""Весна""",-,\N
20205,Ростов-на-Дону,Ярута С.М. ИП,"347332, Ростовская область, Волгодонской р-н, ...",\N
20206,Ростов-на-Дону,Яхонов Б.А. ИП,"346380, Ростовская обл, Красносулинский р-н, Л...",\N
20207,Ростов-на-Дону,Яцкевич Е.Ю.ИП,"344002, Ростовская обл, Ростов-на-Дону г, Шосс...",\N


In [59]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
def combine(row):
    return f"{row['Торг_точка_грязная']} {row['Торг_точка_грязная_адрес']}"

similarity_threshold = 0.85

In [60]:
unique_rows = {}

for index in range(len(data)):
    is_unique = True
    current_name = data.loc[index, 'Торг_точка_грязная']
    current_address = data.loc[index, 'Торг_точка_грязная_адрес']
    current_combined = current_name + '|' + current_address

    # Получаем последние 5 записей из unique_rows
    last_five = list(unique_rows.items())[-5:]

    for key, value in last_five:
        if similar(current_combined, key + '|' + value) >= similarity_threshold:
            is_unique = False
            break

    if is_unique:
        unique_rows[current_name] = current_address

    # Выводим информацию каждые 1000 строк
    if (index + 1) % 1000 == 0:
        print(f"Processed {index + 1} rows.")

print(len(unique_rows))

Processed 1000 rows.
Processed 2000 rows.
Processed 3000 rows.
Processed 4000 rows.
Processed 5000 rows.
Processed 6000 rows.
Processed 7000 rows.
Processed 8000 rows.
Processed 9000 rows.
Processed 10000 rows.
Processed 11000 rows.
Processed 12000 rows.
Processed 13000 rows.
Processed 14000 rows.
Processed 15000 rows.
Processed 16000 rows.
Processed 17000 rows.
Processed 18000 rows.
Processed 19000 rows.
Processed 20000 rows.
10427


In [61]:
outlets_clean = pd.DataFrame(list(unique_rows.items()), columns=['Торг_точка_чистый', 'Торг_точка_чистый_адрес'])
outlets_clean.reset_index(inplace=True)
outlets_clean.rename(columns={'index': 'id'}, inplace=True)

In [62]:
print(data.outlet_clean_id.unique())

['\\N']


In [63]:
data['combined'] = data['Торг_точка_грязная'] + data['Торг_точка_грязная_адрес']

combined_to_id = {row['Торг_точка_чистый'] + row['Торг_точка_чистый_адрес']: row['id'] for index, row in outlets_clean.iterrows()}

def update_outlet_clean_id(row):
    return combined_to_id.get(row['combined'], pd.NA)

data['outlet_clean_id'] = data.apply(update_outlet_clean_id, axis=1)

data.drop(columns=['combined'], inplace=True)
outlets_clean.drop(['id', 'Торг_точка_чистый'], axis=1, inplace=True)


outlets_clean.to_csv('outlets_clean.csv', index=False)
data.to_csv('loaded.csv', index=False)

In [64]:
print(data.outlet_clean_id.unique())

[<NA> 1 4 ... 10424 10425 10426]


In [65]:
outlets_clean.head(3)

,Торг_точка_чистый_адрес
0,"п.Глубокий, ул.Артёма 141"
1,-
2,"346385, Ростовская обл, Красносулинский р-н, Г..."


In [66]:
data.head(3)

,Город дистрибьютора,Торг_точка_грязная,Торг_точка_грязная_адрес,outlet_clean_id
0,Ростов-на-Дону,ИП Гаврилов С.А.,"п.Глубокий, ул.Артёма 44",<NA>
1,Ростов-на-Дону,ИП Малышева,-,1
2,Ростов-на-Дону,Абасян А.П. ИП,он же,<NA>
